In [3]:
# for each scRNA-seq robj
rObj = tiss

In [5]:
geneinfo = read.table("/publicdata/gencode_v19_20151104/gene_info.tsv", header = T)

In [4]:
cts = unique(Idents(object = rObj))

In [6]:
markers = list()

for( celltype in cts){

    celltype_name = gsub(" " , "_", celltype)
    assign(paste(celltype_name, "markers", sep = "_"), FindMarkers(human_skin, ident.1 = celltype, ident.2 = NULL, min.pct = 0.25))        
    markers[[paste(celltype_name, "markers", sep = "_")]] = paste(celltype_name, "markers", sep = "_")

}

In [ ]:
sig.df = data.frame(genes = NA, celltype = NA)


for( i in seq(1, length(markers))){
    
    ct = str_split_fixed(names(markers[i]), "_marke", 2)[,1]
    temp = as.data.frame(markers[[i]])
    temp = temp[order(temp$p_val_adj), ]
    temp = temp[temp$avg_logFC > 0.25 & temp$p_val_adj < 0.05, ]
    
    if(nrow(temp) > 200){ genes2add = rownames(temp[1:200,])}
    else{ genes2add = rownames(temp[1:nrow(temp),]) }

    sig.df.temp = data.frame(genes = genes2add, celltype = ct)
    sig.df = rbind(sig.df, sig.df.temp)

}

sig.df = sig.df[2:nrow(sig.df),]

In [ ]:
sig.df$geneid = geneinfo[match(sig.df$genes, geneinfo$gene_name), "gene_id"]
sig.df$dup = duplicated(sig.df$genes)
sig.df = sig.df[sig.df$dup == F, ]

In [ ]:
sig.avg_exp = AverageExpression(rObj, features = unique(sig.df$genes))
sig.avg_exp = sig.avg_exp$RNA
sig.avg_exp$gene_id = geneinfo[match(rownames(sig.avg_exp), geneinfo$gene_name), "gene_id"]
rownames(sig.avg_exp) <- NULL
signature_gene_matrix = sig.avg_exp[, c(ncol(sig.avg_exp), 1:(ncol(sig.avg_exp) - 1))]